<a href="https://colab.research.google.com/github/PhoebeLu1011/1132code/blob/main/hw5_%E9%9A%A8%E5%A0%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1MiEy6AJogQJ3irjH2Q9O8eHepAdLhBK4egjkzU-9_8Q/edit?usp=sharing').sheet1

In [ ]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1MiEy6AJogQJ3irjH2Q9O8eHepAdLhBK4egjkzU-9_8Q/edit?usp=sharing').get_worksheet(0)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [ ]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [ ]:
df.head()

,num,id,comment,date
0,1,ronny1020,秒選仙台，福岡超沒東西逛,04/12 13:52
1,2,esho,比五月溫度雨量 仙台雙贏 九州開始熱而且雨比較多,04/11 21:02
2,3,jefflyc2002,喜歡都市方便就是福岡，想人群少一點就是仙台,04/11 0:12
3,4,mvpdirk712,博多方便但人也變超多 去仙台吧,04/10 23:36
4,5,aheartcool,兩個都去過 仙台比較喜歡 最愛東北了,04/10 23:02


In [ ]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=cba81809fcb70758db0ab22c55b86f68ef491152f8e609e6be042d2b94606a40
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [ ]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [ ]:
for _, row in dicts.iterrows():
    word = row['comment']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.266 seconds.
DEBUG:jieba:Loading model cost 1.266 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [ ]:
df['Chinese_Content'] = df['comment'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [ ]:
df['Tokenized']

,Tokenized
0,"[秒, 選, 仙台, 福岡超, 沒東西, 逛]"
1,"[比, 五月, 溫度, 雨量, 仙台, 雙贏, 九州, 開始熱, 而且, 雨, 比, 較, 多]"
2,"[喜歡, 都市, 方便, 就是, 福岡想, 人群, 少一點, 就是, 仙台]"
3,"[博多, 方便, 但, 人, 也, 變超, 多, 去, 仙台, 吧]"
4,"[兩個, 都, 去過, 仙台, 比較, 喜歡, 最愛東北, 了]"
5,"[選, 仙台, 說, 不定, 還有, 機會, 看到, 一點, 櫻花]"
6,"[九州, 剛賞, 櫻回來, 不要, 去, 那, 幾個, 很, 有名, 的, 其, 實人, 沒..."
7,"[前陣子, 去, 了, 秋保, 交通, 是, 真的, 方便, 各家, 旅館, 都, 有, 仙..."
8,"[仙台, 較, 舒適, 觀光客, 較, 少, 北九州, 韓國人, 多, 到, 誇張]"
9,"[感謝, 大家, 建議, 目前, 看來, 可能, 比較, 偏向, 去, 仙台, 住宿, 跟,..."


In [ ]:
import re
import pandas as pd

# 定義角色判定的作者名單
author = ['Vincient']  # 根據先前提供的名單

# 定義角色判定規則的函數
def determine_role(row):
    comment = row['comment']
    user_id = row['id']
    if user_id in author:
        return '作者'
    mention_sendai = bool(re.search(r'(仙台|秋保)', comment))
    mention_fukuoka = bool(re.search(r'(福岡|北九州|博多)', comment))
    if mention_sendai and mention_fukuoka:
        return '中立'
    elif mention_sendai:
        return '偏向仙台'
    elif mention_fukuoka:
        return '偏向博多'
    else:
        return '未知'

# 是否提到「九州」的判斷函數
def mention_japan(comment):
    return '是' if re.search(r'九州', comment) else '否'


df['Opinion'] = df.apply(determine_role, axis=1)
df['MentionJapan'] = df['comment'].apply(lambda x: mention_japan(str(x)))


In [ ]:
import jieba

# 關鍵詞清單
location_keywords = ['仙台', '東北', '福岡', '九州', '北九州', '博多', '太宰府', '由布院', '秋保']
organization_keywords = ['旅館', '接駁車', '航空', '機票', 'JR', '飯店']
person_pattern = r'[\u4e00-\u9fa5]{2,4}'

# 抽取實體的函數
def extract_entities(text):
    entities = []
    words = jieba.lcut(text)
    for word in words:
        if word in location_keywords:
            entities.append({"text": word, "type": "Location"})
        elif word in organization_keywords:
            entities.append({"text": word, "type": "Organization"})
        elif re.fullmatch(person_pattern, word):
            entities.append({"text": word, "type": "Person"})
    return entities

df['Entities'] = df['comment'].apply(lambda x: extract_entities(str(x)))


In [ ]:
df['Entities']

,Entities
0,"[{'text': '仙台', 'type': 'Location'}, {'text': ..."
1,"[{'text': '五月', 'type': 'Person'}, {'text': '溫..."
2,"[{'text': '喜歡', 'type': 'Person'}, {'text': '都..."
3,"[{'text': '博多', 'type': 'Location'}, {'text': ..."
4,"[{'text': '兩個', 'type': 'Person'}, {'text': '去..."
5,"[{'text': '仙台', 'type': 'Location'}, {'text': ..."
6,"[{'text': '九州', 'type': 'Location'}, {'text': ..."
7,"[{'text': '前陣子', 'type': 'Person'}, {'text': '..."
8,"[{'text': '仙台', 'type': 'Location'}, {'text': ..."
9,"[{'text': '感謝', 'type': 'Person'}, {'text': '大..."


In [ ]:
df['Opinion']

,Opinion
0,中立
1,偏向仙台
2,中立
3,中立
4,偏向仙台
5,偏向仙台
6,偏向仙台
7,偏向仙台
8,中立
9,作者


In [ ]:
df

,num,id,comment,date,Chinese_Content,Tokenized,Role,MentionJapan,Opinion
0,1,ronny1020,秒選仙台，福岡超沒東西逛,04/12 13:52,秒選仙台福岡超沒東西逛,"[秒, 選, 仙台, 福岡超, 沒東西, 逛]",中立,否,中立
1,2,esho,比五月溫度雨量 仙台雙贏 九州開始熱而且雨比較多,04/11 21:02,比五月溫度雨量仙台雙贏九州開始熱而且雨比較多,"[比, 五月, 溫度, 雨量, 仙台, 雙贏, 九州, 開始熱, 而且, 雨, 比, 較, 多]",偏向仙台,是,偏向仙台
2,3,jefflyc2002,喜歡都市方便就是福岡，想人群少一點就是仙台,04/11 0:12,喜歡都市方便就是福岡想人群少一點就是仙台,"[喜歡, 都市, 方便, 就是, 福岡想, 人群, 少一點, 就是, 仙台]",中立,否,中立
3,4,mvpdirk712,博多方便但人也變超多 去仙台吧,04/10 23:36,博多方便但人也變超多去仙台吧,"[博多, 方便, 但, 人, 也, 變超, 多, 去, 仙台, 吧]",中立,否,中立
4,5,aheartcool,兩個都去過 仙台比較喜歡 最愛東北了,04/10 23:02,兩個都去過仙台比較喜歡最愛東北了,"[兩個, 都, 去過, 仙台, 比較, 喜歡, 最愛東北, 了]",偏向仙台,否,偏向仙台
5,6,oowoo,選仙台~說不定還有機會看到一點櫻花,04/11 10:01,選仙台說不定還有機會看到一點櫻花,"[選, 仙台, 說, 不定, 還有, 機會, 看到, 一點, 櫻花]",偏向仙台,否,偏向仙台
6,7,cyminum,九州剛賞櫻回來，不要去那幾個很有名的其實人沒有到很誇張（由布院太宰府等），兩個地方購物都蠻方...,04/11 7:49,九州剛賞櫻回來不要去那幾個很有名的其實人沒有到很誇張由布院太宰府等兩個地方購物都蠻方便的仙台...,"[九州, 剛賞, 櫻回來, 不要, 去, 那, 幾個, 很, 有名, 的, 其, 實人, 沒...",偏向仙台,是,偏向仙台
7,8,nnofance,前陣子去了秋保，交通是真的方便，各家旅館都有仙台出發的接駁車，然後基本都是當地人…,04/12 0:19,前陣子去了秋保交通是真的方便各家旅館都有仙台出發的接駁車然後基本都是當地人,"[前陣子, 去, 了, 秋保, 交通, 是, 真的, 方便, 各家, 旅館, 都, 有, 仙...",偏向仙台,否,偏向仙台
8,9,feelan,仙台較舒適 觀光客較少 北九州韓國人多到誇張,04/10 22:07,仙台較舒適觀光客較少北九州韓國人多到誇張,"[仙台, 較, 舒適, 觀光客, 較, 少, 北九州, 韓國人, 多, 到, 誇張]",中立,是,中立
9,10,Vincient,感謝大家建議 目前看來可能比較偏向去仙台，住宿跟溫泉來回交通也要先確定好再來機票,04/10 22:28,感謝大家建議目前看來可能比較偏向去仙台住宿跟溫泉來回交通也要先確定好再來機票,"[感謝, 大家, 建議, 目前, 看來, 可能, 比較, 偏向, 去, 仙台, 住宿, 跟,...",作者,否,作者


In [ ]:
from collections import Counter

Japan_counts = Counter(df['MentionJapan'])

In [ ]:
Japan_counts

Counter({'否': 7, '是': 3})

In [ ]:
import plotly.express as px

# Sorting the school counts in descending order
sorted_jp_counts = sorted(Japan_counts.items(), key=lambda x: x[1], reverse=True)
sorted_jp_counts_dict = dict(sorted_jp_counts)

# Creating a sorted DataFrame for the bar chart
sorted_school_df = pd.DataFrame(list(sorted_jp_counts_dict.items()), columns=["MentionJapan", "Count"])

# Plotting the sorted school distribution
fig_sorted = px.bar(sorted_school_df, x='MentionJapan', y='Count',
                    title="是否提及九州",
                    labels={"MentionJapan": "MentionJapan", "Count": "Number of Mentions"},
                    text='Count')

fig_sorted.update_traces(texttemplate='%{text}', textposition='outside')
fig_sorted.update_layout(xaxis=dict(title="是否提及九州"), yaxis=dict(title="Number of Mentions"))

fig_sorted.show()

In [ ]:
# 計算每個 Account 的留言次數
account_message_count = df.groupby('id').size().reset_index(name='Message_Count')

# 合併學校名稱到 account_message_count
account_school = df[['id', 'Opinion']].drop_duplicates()

# 合併留言次數與學校名稱
result = account_message_count.merge(account_school, on='id')

In [ ]:
result.describe()

,Message_Count
count,10.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [ ]:
result

,id,Message_Count,Opinion
0,Vincient,1,作者
1,aheartcool,1,偏向仙台
2,cyminum,1,偏向仙台
3,esho,1,偏向仙台
4,feelan,1,中立
5,jefflyc2002,1,中立
6,mvpdirk712,1,中立
7,nnofance,1,偏向仙台
8,oowoo,1,偏向仙台
9,ronny1020,1,中立


In [ ]:
# Group the data by 'School' and aggregate the message count and accounts
grouped_data = result.groupby('Opinion').agg({
    'id': lambda accounts: ', '.join(map(str, accounts)),  # Convert integers to strings before joining
    'Message_Count': 'sum'
}).reset_index()

# Rename columns for clarity
grouped_data.rename(columns={
    'Opinion': 'Opinion_Name',
    'id': 'ID',
    'Message_Count': 'Total_Messages'
}, inplace=True)

# Sort the grouped data by Total_Messages in descending order
sorted_data = grouped_data.sort_values(by='Total_Messages', ascending=False).reset_index(drop=True)
sorted_data

,Opinion_Name,ID,Total_Messages
0,偏向仙台,"aheartcool, cyminum, esho, nnofance, oowoo",5
1,中立,"feelan, jefflyc2002, mvpdirk712, ronny1020",4
2,作者,Vincient,1


In [ ]:
raw_accounts1 = list(sorted_data[sorted_data['Opinion_Name'] == '偏向仙台']['ID'])

In [ ]:
raw_accounts1

['aheartcool, cyminum, esho, nnofance, oowoo']

In [ ]:
raw_accounts2 = list(sorted_data[sorted_data['Opinion_Name'] == '偏向博多']['ID'])

In [ ]:
raw_accounts2

[]

In [ ]:
raw_accounts3 = list(sorted_data[sorted_data['Opinion_Name'] == '中立']['ID'])
raw_accounts3

['feelan, jefflyc2002, mvpdirk712, ronny1020']

In [ ]:
accounts_to_extract = set(raw_accounts[0].split(', '))

In [ ]:
accounts_to_extract

{'feelan', 'jefflyc2002', 'mvpdirk712', 'ronny1020'}

In [ ]:
filtered_df = df[df['id'].isin(accounts_to_extract)]

In [ ]:
from collections import Counter

Japan_counts2 = Counter(df['Opinion'])

In [ ]:
import plotly.express as px

# Sorting the school counts in descending order
sorted_jp_counts2 = sorted(Japan_counts2.items(), key=lambda x: x[1], reverse=True)
sorted_jp_counts_dict2 = dict(sorted_jp_counts2)

# Creating a sorted DataFrame for the bar chart
sorted_school_df = pd.DataFrame(list(sorted_jp_counts_dict2.items()), columns=["Opinion", "Count"])

# Plotting the sorted school distribution
fig_sorted = px.bar(sorted_school_df, x='Opinion', y='Count',
                    title="仙台 v.s. 博多 旅遊景點選擇",
                    labels={"Opinion": "Opinion", "Count": "Number of Mentions"},
                    text='Count')

fig_sorted.update_traces(texttemplate='%{text}', textposition='outside')
fig_sorted.update_layout(xaxis=dict(title="留言立場"), yaxis=dict(title="Number of Mentions"))

fig_sorted.show()

In [ ]:
filtered_df

,num,id,comment,date,Chinese_Content,Tokenized,Role,MentionJapan,Opinion
0,1,ronny1020,秒選仙台，福岡超沒東西逛,04/12 13:52,秒選仙台福岡超沒東西逛,"[秒, 選, 仙台, 福岡超, 沒東西, 逛]",中立,否,中立
2,3,jefflyc2002,喜歡都市方便就是福岡，想人群少一點就是仙台,04/11 0:12,喜歡都市方便就是福岡想人群少一點就是仙台,"[喜歡, 都市, 方便, 就是, 福岡想, 人群, 少一點, 就是, 仙台]",中立,否,中立
3,4,mvpdirk712,博多方便但人也變超多 去仙台吧,04/10 23:36,博多方便但人也變超多去仙台吧,"[博多, 方便, 但, 人, 也, 變超, 多, 去, 仙台, 吧]",中立,否,中立
8,9,feelan,仙台較舒適 觀光客較少 北九州韓國人多到誇張,04/10 22:07,仙台較舒適觀光客較少北九州韓國人多到誇張,"[仙台, 較, 舒適, 觀光客, 較, 少, 北九州, 韓國人, 多, 到, 誇張]",中立,是,中立


In [ ]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"用繁體中文分析在這個PTT的留言中上對特定選擇，以及不選擇其他的原因，或是中立的原因。此外幫我整理出去兩地旅遊的優缺點：{filtered_df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這份PTT留言數據分析了網友選擇日本仙台或福岡旅遊的原因。  以下整理分析，並列出兩地旅遊的優缺點：

**仙台選擇原因分析：**

* **留言0, 3, 8:**  直接或間接表達了選擇仙台的原因：
    * **舒適且觀光客較少 (留言8):**  相較於其他城市，仙台的環境更舒適，遊客也較少，行程體驗更佳。這點是正向因素，吸引了偏好清靜旅遊體驗的旅客。
    * **人群較少 (留言2, 3):**  與福岡相比，仙台的人群密度較低，更符合部分旅客的偏好。這點也是正向因素。


**福岡選擇原因分析：**

* **留言2, 3:**  指出選擇福岡的原因：
    * **都市方便 (留言2, 3):**  福岡作為大都市，交通和生活機能都較便利，這點吸引偏好便捷都市生活的旅客。這是正向因素。


**不選擇其他城市的原因分析：**

* **留言0:**  直接表示「福岡超沒東西逛」，說明不選擇福岡的原因是缺乏觀光資源，屬於負面因素。
* **留言8:**  提到「北九州韓國人多到誇張」，表示不選擇北九州的原因是遊客組成與自身期待不符，屬於負面因素(這點並非針對福岡，但可以推測，若北九州韓國遊客多，福岡可能也存在相似問題，但程度未知)。


**中立意見：**

留言 2 和 3  提供了客觀比較，指出仙台和福岡各自的優缺點，而非單純表達偏好。  這屬於中立的觀點，幫助理解不同旅客的需求。


**仙台與福岡旅遊優缺點整理：**

**仙台:**

* **優點:**  環境舒適、遊客較少、人群密度低、清靜宜人。
* **缺點:**  資料中沒有直接提及缺點，但可以推測可能缺乏大型購物中心或娛樂設施，較不適合偏好都市型娛樂的旅客。


**福岡:**

* **優點:**  都市機能便利、交通方便、生活機能完善。
* **缺點:**  人潮擁擠，觀光資源可能相對較少(依留言0)。


**總結:**

選擇仙台還是福岡，取決於旅客的個人偏好。 喜歡清幽環境、人群較少的旅客會傾向選擇仙台；而喜歡都市便利性、交通和生活機能完善的旅客則會偏好福岡。  以上分析僅基於提供的PTT留言，實際情況可能更為複雜。



In [ ]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 36 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 0s (36.7 MB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
stop_word = set(['，', '就是','最近', '雖然', '感到', '而且', '晚上', '常常', '看來', '幾個', '和', '選' , '偶爾'])
word_counts = Counter(all_words)
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_word}
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['comment', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

In [ ]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['comment'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [ ]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請幫我從以下一個一個(10個都要)顯示留言地點、時間(旅遊時間，例如前陣子就寫前陣子，不要寫留言時間，不知道就寫未知)、為甚麼選擇此方向的統整，有點想要向ner那樣：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

## 仙台 vs 福岡 旅遊抉擇分析：NER 風格

以下整理了 10 則留言，分析使用者選擇仙台或福岡的原因：


| num | id          | 地點           | 時間       | 選擇方向原因                                                                                                                                                                 | 偏好 |
|-----|-------------|-----------------|-------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------|
| 0   | ronny1020   | 仙台, 福岡       | 未知       | 秒選仙台，認為福岡沒東西逛。                                                                                                                                                         | 仙台 |
| 1   | esho        | 仙台, 九州       | 五月       | 比較五月溫度雨量，認為仙台雙贏，九州開始熱而且雨比較多。                                                                                                                            | 仙台 |
| 2   | jefflyc2002 | 福岡, 仙台       | 未知       | 